<a href="https://colab.research.google.com/github/Armandohaj/Proyecto-Talleretapa/blob/main/proyecto.final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: When **using** this notebook, you must select a GPU Hardware Accelerator instead of CPU. Go to Runtime->Change Runtime Type and select any option that uses GPU.

# Dependencies

In [3]:
!pip install face_recognition # Install face_recognition package using pip
import face_recognition
import cv2 # Import OpenCV
import numpy as np
from matplotlib import pyplot as plt # pyplot
from pickle import load, dump

# ipywidgets packages
import ipywidgets as widgets
from IPython.display import display, clear_output

# Live Streaming dependencies
from IPython.display import Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import cv2
import matplotlib.pyplot as plt

# Define la variable I (asegúrate de darle un valor apropiado)
i = 1
# Ruta de la imagen
img_path = '/content/drive/MyDrive/Colab Notebooks/Images/' + 'img' + str(i) + '.jpg'

# Aquí debes tener una imagen válida en la variable 'img' antes de guardarla
# cv2.imwrite(img_path, img)

def show_image(image_path):
    """ Esta función muestra una imagen.
      Argumentos:
          image_path {str} -- Path de la imagen.
    """
    img = cv2.imread(image_path)
    img_cvt = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_cvt)
    plt.show()

# Define las coordenadas x1, y1, x2, y2
x1 = 100
y1 = 100
x2 = 200
y2 = 200

# Lista para guardar las rutas de las imágenes
img_list = []

# Agrega la ruta de la imagen a la lista
img_list.append(img_path)

def show_image_with_square(image_path, faces):
    """ Esta función muestra una imagen con un cuadro en cada rostro.
      Argumentos:
          image_path {str} -- Path de la imagen.
          faces {list} -- Lista de diccionarios con las coordenadas de los rostros.
    """
    img = cv2.imread(image_path)
    im2Display = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imTemp = im2Display.copy()
    for face in faces:
        x1 = face['x1']
        y1 = face['y1']
        x2 = face['x2']
        y2 = face['y2']
        imTemp = cv2.rectangle(imTemp, (x1, y1), (x2, y2), (0, 255, 0), 2)
    plt.imshow(imTemp)
    plt.show()

# Llama a las funciones según sea necesario
# show_image(img_path)
# face_rectangle = {'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2}
# show_image_with_square(img_path, [face_rectangle])



# Utils

## In-memory data

In [29]:
registered_faces = {}
registered_professors = {}
registered_students = {}
registered_miscellaneous  = []
registered_name_professor = []
lista_estudiantes = []
lista_profesores = []
numero_imagen = 1


## Real time streaming

In [6]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 800, 600);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 800; //video.videoWidth;
      captureCanvas.height = 600; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

In [8]:
# JavaScript code to stop the webcam stream
js_stop_stream = """
    const stream = video.srcObject;
    const tracks = stream.getTracks();

    tracks.forEach(function(track) {
        track.stop();
    });

    video.srcObject = null;
    div.style.display = 'none';  // Hide the container
"""

# Function to stop the video streaming
def stop_video_stream():
    display(Javascript(js_stop_stream))

# Register Faces

In [9]:
from ipywidgets.widgets import widget
import pickle

uploaded_image_data = ""
# Create a text input widget for the name
name_input = widgets.Text(
    placeholder='Ingrese su nombre',
    description='Nombre:',
    disabled=False
)

# Create a file upload widget
upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Cargar Imagen'
)

or_text = widgets.HTML(
    value=" o "
)

# Create a button to trigger face registration
register_button = widgets.Button(description='Registrar Cara')

# Create a button to trigger image capture
capture_button = widgets.Button(description='Capturar Imagen')

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

# Initialize variables to store the uploaded image and name
uploaded_image_data = None
registered_name_professor = None
registered_name_student = None
registered_miscellaneous = None


# Import necessary libraries (ensure you have them installed)
import numpy as np
import cv2
import face_recognition

# Define a function to handle image upload
def handle_upload(change):

    global numero_imagen
    global uploaded_image_data

    uploaded_filename = list(upload_button.value.keys())[0]
    uploaded_image_data = upload_button.value[uploaded_filename]['content']

    with output:
        clear_output()
        display(widgets.Image(value=uploaded_image_data))

    # Assuming uploaded_image_data is binary image data in bytes
    image_array = np.frombuffer(uploaded_image_data, np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    # Convert the image to RGB format
    uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #print(numero_imagen)
    cv2.imwrite('/content/drive/MyDrive/imagenes/'+ 'numero'+str(numero_imagen)+'.jpg',image )
    numero_imagen += 1

# Define a function to handle image capture
def handle_capture(button):
    global numero_imagen
    global uploaded_image_data
    # Start streaming video from the webcam
    video_stream()
    js_reply = video_frame("Capturando...", "")
    stop_video_stream()  # Stop the video stream after capturing the frame

    if not js_reply:
        with output:
            clear_output()
            print("Error al capturar la imagen!")

    # Convert JS response to OpenCV Image

    uploaded_image_data = js_to_image(js_reply["img"])
    #trabar esto para los archivos txt
    cv2.imwrite('/content/drive/MyDrive/imagenes/'+ 'numero'+str(numero_imagen)+'.jpg',uploaded_image_data )
    numero_imagen += 1

    with output:
        clear_output()
        # Encode the captured frame as PNG
        _, buffer = cv2.imencode('.png', uploaded_image_data)

        # Convert the encoded image to bytes
        image_bytes = buffer.tobytes()
        display(widgets.Image(value=image_bytes))

# Define a function to check if a face is already registered
def check_if_face_registered(encoding):
    global registered_faces
    for face_name, face_encoding in registered_faces.items():
        print(encoding,"-",face_encoding.get("encoding"))
        matches = face_recognition.compare_faces([face_encoding.get("encoding")], encoding)
        if any(matches):
            return True
    return False

# Define a function to handle face registration
def handle_register(button):

    global registered_faces
    global registered_name_professor
    global registered_name_student
    global registered_name_miscellaneous

    if uploaded_image_data is not None:
        registered_name = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
          with output:
              clear_output()
              print("Esta cara ya está registrada.")
              return

        with output:
            clear_output()
            print(f"Cara registrada para {registered_name}.")


            # Store the face encoding along with the role
            registered_faces[registered_name] = {
                'encoding': face_encodings[0],
                'role': role_dropdown.value  # Use the selected role
            }

# Create a dropdown widget for role selection
role_dropdown = widgets.Dropdown(
    options=['Estudiante', 'Profesor', 'Misceláneo'],
    description='Rol:',
    disabled=False
)

# Set up the widget event handlers
upload_button.observe(handle_upload, names='value')
capture_button.on_click(handle_capture)
register_button.on_click(handle_register)

# Create a button to trigger face registration for professors
register_button_professor = widgets.Button(description='Registrar Profesor')

# Define a function to handle face registration for professors
registered_professors = {}

def handle_register_professor(button):
    global registered_professors
    global registered_faces

    if uploaded_image_data is not None:
        # Obtener el nombre del profesor desde el campo de entrada
        registered_name_professor = name_input.value

        # Detectar caras en la imagen cargada
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # Si no se detecta ninguna cara, mostrar una ventana emergente
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # Si se detecta más de una cara, mostrar una ventana emergente
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return

        # Obtener las codificaciones de las caras
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Comprobar si la cara ya está registrada
        if check_if_face_registered(face_encodings[0]):
            with output:
                clear_output()
                print("Esta cara ya está registrada.")
                return

        # Recopilar información adicional sobre el profesor
        identificacion = widgets.Text(description="Identificacion: ")
        fecha_de_nacimiento = widgets.Text(description="Nacimiento: ")
        edad = widgets.text(description ="edad")
        nacionalidad = widgets.Text(description="Nacionalidad: ")
        telefono = widgets.Text(description="Teléfono: ")
        celular = widgets.Text(description="Celular: ")
        email = widgets.Text(description="Email: ")
        direccion = widgets.Text(description="Dirección: ")
        especialidad = widgets.Text(description="Especialidad: ")
        años_dando_clases = widgets.Text(description="Años dando clases: ")
        genero = widgets.Text(description="Genero: ")

        lista_profesores = []

        def guardar_handler(_):
            # Almacenar la codificación de la cara junto con la información del profesor
            profesor = {
                'nombre': registered_name_professor,
                'encoding': face_encodings[0],
                'role': 'Profesor',
                'informacion_adicional': {
                    'identificacion': identificacion.value,
                    'fecha_de_nacimiento': fecha_de_nacimiento.value,
                    'edad': edad.value
                    'nacionalidad': nacionalidad.value,
                    'telefono': telefono.value,
                    'celular': celular.value,
                    'email': email.value,
                    'direccion': direccion.value,
                    'especialidad': especialidad.value,
                    'años_dando_clases': años_dando_clases.value,
                    'genero': genero.value,
                }
            }

            # Add the professor to the dictionary of registered professors
            registered_professors[registered_name_professor] = profesor

            # Optional: Save the dictionary of registered miscellaneous individuals to a file
            with open("/content/drive/MyDrive/informacion/archivoprofesor.txt", 'a') as archivo:
                archivo.write(str(profesor))

            print(f"Información registrada para {registered_name_professor} (Profesor).")

        guardar_professor_button = widgets.Button(description='Guardar información')
        guardar_professor_button.on_click(guardar_handler)
        display(registered_name_professor, identificacion, fecha_de_nacimiento,edad, nacionalidad, telefono, celular, email,
                direccion, especialidad, años_dando_clases, genero, guardar_professor_button)


register_button_students = widgets.Button(description='Registrar Estudiante')

# Define a function to handle face registration for students
registered_students = {}

def handle_register_student(button):
    global registered_students
    global registered_faces
    global registered_name_student

    if uploaded_image_data is not None:
        registered_name_student = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
            with output:
                clear_output()
                print("Esta cara ya está registrada.")
                return

        print(f"Cara registrada para {registered_name_student} (estudiante).")

        # Recopilar información adicional sobre el estudiante
        identificacion = widgets.Text(description="Identificacion: ")
        nombre = widgets.Text(description="Nombre: ")
        fecha_de_nacimiento = widgets.Text(description="Nacimiento: ")
        edad = widgets.text(description ="edad")
        nacionalidad = widgets.Text(description="Nacionalidad: ")
        telefono = widgets.Text(description="Teléfono: ")
        celular = widgets.Text(description="Celular: ")
        email = widgets.Text(description="Email: ")
        direccion = widgets.Text(description="Dirección: ")
        carrera = widgets.Text(description="Carrera: ")
        pasatiempos = widgets.Text(description="Pasatiempos: ")
        signo_zodiacal = widgets.Text(description="Signo zodiacal: ")

        def guardar_student(_):
            # Almacenar la codificación de la cara junto con la información del estudiante
            student = {
                'nombre': registered_name_student,
                'encoding': face_encodings[0],
                'role': 'Student',
                'informacion_adicional': {
                    'identificacion': identificacion.value,
                    'nombre': nombre.value,
                    'fecha_de_nacimiento': fecha_de_nacimiento.value,
                    'edad':edad.value,
                    'nacionalidad': nacionalidad.value,
                    'telefono': telefono.value,
                    'celular': celular.value,
                    'email': email.value,
                    'direccion': direccion.value,
                    'carrera': carrera.value,
                    'pasatiempos': pasatiempos.value,
                    'signo_zodiacal': signo_zodiacal.value
                }
            }

            # Add the student to the dictionary of registered students
            registered_students[registered_name_student] = student

            # Optional: Save the dictionary of registered miscellaneous individuals to a file
            with open("/content/drive/MyDrive/Informacion/archivoestudiantes.txt", 'a') as archivo:
                archivo.write(str(student))

            print(f"Información registrada para {registered_name_student} (estudiante).")

        guardar_student_button = widgets.Button(description='Guardar información')
        guardar_student_button.on_click(guardar_student)
        display(identificacion, nombre, fecha_de_nacimiento,edad, nacionalidad, telefono, celular, email, direccion, carrera,
                pasatiempos, signo_zodiacal, guardar_student_button)

register_button_miscellaneo = widgets.Button(description='Registrar Micelanio')
# Define a function to handle face registration for micelaneo
registered_miscellaneous = {}

def handle_register_miscellaneous(button):
    global registered_miscellaneous
    global registered_faces
    global registered_name_miscellaneous

    if uploaded_image_data is not None:
        registered_name_miscellaneous = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
            with output:
                clear_output()
                print("Esta cara ya está registrada.")
                return

        with output:
            clear_output()
            print(f"Cara registrada para {registered_name_miscellaneous} (miscellaneous).")

            # Store the face encoding along with the role
            registered_faces[registered_name_miscellaneous] = {
                'encoding': face_encodings[0],
                'role': 'Miscellaneous'
            }

        # Recopilar información adicional sobre el miscellaneous

        identificacion = widgets.Text(description="Identificacion: ")
        nombre = widgets.Text(description="Nombre: ")
        fecha_de_nacimiento = widgets.Text(description="Nacimiento: ")
        edad = widgets.text(description = "edad")
        nacionalidad = widgets.Text(description="Nacionalidad: ")
        telefono = widgets.Text(description="Teléfono: ")
        celular = widgets.Text(description="Celular: ")
        email = widgets.Textarea(description="Email: ")
        direccion = widgets.Text(description="Dirección: ")
        turno = widgets.Text(description="Turno: ")
        habilidad = widgets.Text(description="Habilidad: ")
        estado_civil = widgets.Text(description="Estado civil: ")

        def guardar_miscellaneous(_):
            # Almacenar la codificación de la cara junto con la información del miscellaneous
            miscellaneous_info = {
                'nombre': registered_name_miscellaneous,
                # 'encoding': face_encodings[0],
                'role': 'Miscellaneous',
                'informacion_adicional': {
                    'identificacion': identificacion.value,
                    'nombre': nombre.value,
                    'fecha_de_nacimiento': fecha_de_nacimiento.value,
                    'edad': edad.value,
                    'nacionalidad': nacionalidad.value,
                    'telefono': telefono.value,
                    'celular': celular.value,
                    'email': email.value,
                    'direccion': direccion.value,
                    'turno': turno.value,
                    'habilidad': habilidad.value,
                    'estado_civil': estado_civil.value
                }
            }

            # Add the miscellaneous person to the dictionary of registered miscellaneous individuals
            registered_miscellaneous[registered_name_miscellaneous] = miscellaneous_info

            # Optional: Save the dictionary of registered miscellaneous individuals to a file
            with open("/content/drive/MyDrive/Informacion/archivomiscelaneos.txt", 'a') as archivo:
                archivo.write(str(miscellaneous_info))

            print(f"Información registrada para {registered_name_miscellaneous} (miscellaneous).")

        guardar_miscellaneous_button = widgets.Button(description='Guardar información')
        guardar_miscellaneous_button.on_click(guardar_miscellaneous)
        display(identificacion, nombre, fecha_de_nacimiento,edad, nacionalidad, telefono, celular, email, direccion, turno,
                habilidad, estado_civil, guardar_miscellaneous_button)


# Set up the widget event handler for registering professors
register_button_professor.on_click(handle_register_professor)
register_button_students.on_click(handle_register_student)
register_button_miscellaneo.on_click(handle_register_miscellaneous)
# Create a container for the widgets
container = widgets.VBox([name_input, role_dropdown, widgets.HBox([upload_button, or_text, capture_button]),
                          widgets.HBox([register_button, or_text, register_button_professor, or_text, register_button_students, or_text, register_button_miscellaneo]), output])

# Display the container
display(container)


In [ ]:
print(registered_professors)

In [ ]:
print(registered_faces)

In [ ]:
import pickle

# Abre el archivo en modo lectura
with open("/content/drive/MyDrive/imagenes/", "rb") as f:
    try:
        # Carga los datos almacenados en el archivo
        data = pickle.load(f)

        # Ahora puedes imprimir o procesar los datos según tus necesidades
        for nombre, info in data.items():
            print("Nombre:", nombre)
            print("Role:", info['role'])

            # Accede a la información adicional
            informacion_adicional = info.get('informacion_adicional', {})
            print("Información Adicional:")
            print("Identificación:", informacion_adicional.get('identificacion'))
            print("Nombre:", informacion_adicional.get('nombre'))
            print("Fecha de Nacimiento:", informacion_adicional.get('fecha_de_nacimiento'))
            # Agrega más campos según sea necesario

    except EOFError:
        print("El archivo está vacío o ha llegado al final.")
    except Exception as e:
        print("Se produjo un error al leer el archivo:", str(e))


In [13]:
def arch_binario(archivo_encriptado):
  with open(archivo_encriptado,"rb") as archivo_ens:
    datos_encriptados = archivo_ens.read()
    print(datos_encriptados)



In [ ]:
ruta = "/content/drive/MyDrive/Informacioncurso/infocurso.txt"
arch_binario(ruta)

In [ ]:

print(registered_professors)
print(registered_faces)

### informacion del aula

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Lista para almacenar diccionarios de aula
aula_informacion = []

# Crear widgets
numero_de_aula = widgets.Text(description="Número: ")
carnet_miscelaneo = widgets.Text(description="Carnet: ")
capacidad = widgets.Text(description = "Capacidad: ")
aire = widgets.Text(description = "Aire: ")

guardar_button = widgets.Button(description="Guardar Información")
print("informacion del aula: ")

# Función para guardar la información
import os
import pickle

def guardar_aula(_):
    global aula_informacion
    aula_info = {
        'numero': numero_de_aula.value,
        'carnet': carnet_miscelaneo.value,
        'capacidad': capacidad.value,
        'aire': aire.value
    }

    # Intenta cargar información previa si existe
    if os.path.exists("/content/drive/MyDrive/Informacionaula/archivoaula.txt"):
        with open("/content/drive/MyDrive/Informacionaula/archivoaula.txt", "rb") as f:
            try:
                aula_informacion = pickle.load(f)
            except pickle.UnpicklingError:
                aula_informacion = []
    else:
        aula_informacion = []

    # Agrega la nueva información del aula a la lista existente
    aula_informacion.append(aula_info)

    # Optional: Save the dictionary of registered miscellaneous individuals to a file
    with open("/content/drive/MyDrive/Informacionaula/archivoaula.txt", 'a') as archivo:
        archivo.write(str(aula_info))


    print("Información del aula guardada con éxito.")



# Boton para guardar
guardar_button.on_click(guardar_aula)

# Posicion de los widgets
form = widgets.VBox([numero_de_aula, carnet_miscelaneo, capacidad, aire, guardar_button])

# imprimir
display(form)


In [ ]:
print(aula_informacion)

### informacion del curso

In [ ]:
import ipywidgets as widgets
from IPython.display import display
global informacion_curso
# lista para guardar la informacion del curso
informacion_curso = []
print("informacion del curso: ")

#widgets
codigo_de_curso = widgets.Text(description = "Codigo: ")
nombre_de_curso = widgets.Text(description = "Nombre: ")
periodo = widgets.Text(description = "Periodo: ")
carrera_al_que_pertenece = widgets.Text(description = "Carrera: ")
horario = widgets.Text(description = "Horario: ")
aula = widgets.Text(description = "aula: ")
guardar_button = widgets.Button(description = "Guardar informacion")

# funcion para almacenar la informacion del curso
def guardar_curso(_):
    global informacion_curso
    info_curso = {
      'codigo': codigo_de_curso.value,
      'nombre': nombre_de_curso.value,
      'periodo': periodo.value,
      'carrera': carrera_al_que_pertenece.value,
      'lista_estudiantes' : lista_estudiantes,
      'profesor_lista': lista_profesores,
      'horario': horario.value,
      'aula': aula.value,
    }

    # Intenta cargar información previa si existe
    with open("/content/drive/MyDrive/Informacioncurso/archivocurso.txt", "rb") as f:
          try:
              informacion_curso = pickle.load(f)
          except pickle.UnpicklingError:
              informacion_curso = []


    # Agrega la nueva información del curso a la lista existente
    informacion_curso.append(info_curso)

    # Optional: Save the dictionary of registered miscellaneous individuals to a file
    with open("/content/drive/MyDrive/Informacioncurso/archivocurso.txt", 'a') as archivo:
        archivo.write(str(informacion_curso))

    print("Información del curso guardada con éxito.")

def obtener_estudiantes(curso, estudiantes):
    for estudiante, informacion in estudiantes.items():
        if curso.get("identificacion") == informacion.get("identificacion"):
            return informacion
    return None


def obtener_profesor(curso, profesores):
    for profesor, informacion in profesores.items():
        if curso.get("identificacion") == informacion.get("identificacion"):
            return informacion
    return None



# Boton para guardar
guardar_button.on_click(guardar_curso)

# Posicion de los widgets
form = widgets.VBox([codigo_de_curso, nombre_de_curso, periodo, carrera_al_que_pertenece, horario, aula, guardar_button])

# imprimir
display(form)




informacion del curso: 


In [ ]:
icape = informacion_curso[0]
print(obtener_profesor(icape, registered_professors))
print(obtener_estudiantes(icape, registered_students))

In [ ]:
print(informacion_curso[0])

# Recognize Faces

In [ ]:
import face_recognition
import cv2
from datetime import datetime
import matplotlib.pyplot as plt

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''
i = 0

match_info = []
path_cuadro = []

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])
    # Get the face locations from the frame
    face_locations = face_recognition.face_locations(frame)

    img_path = '/content/drive/MyDrive/imagenlista/' + 'frame' + str(i) + '.jpg'
    cv2.imwrite(img_path, frame)

    # For each face in the frame
    for (top, right, bottom, left) in face_locations:
        # Draw a rectangle around the face
        cv2.rectangle(
            frame,
            (left, top),
            (right, bottom),
            (0, 0, 255),
            2,
        )
        # Get the face encoding of the face
        face_encoding = face_recognition.face_encodings(frame, [(top, right, bottom, left)])[0]

        # Compare the face encoding to the faces in the database (registered_students)
        matches = face_recognition.compare_faces(list(registered_students.values()), face_encoding)

        # If there is a match
        if True in matches:
            faces_list = []
            for face_location in face_locations:  # Itera sobre las coordenadas de todas las caras
                top, right, bottom, left = face_location
                img_with_rectangle = cv2.rectangle(frame.copy(), (left, top), (right, bottom), (0, 255, 0), 2)
                face_rectangle = {'x1': left, 'y1': top, 'x2': right, 'y2': bottom}
                faces_list.append(face_rectangle)
                path_cuadro.append([img_path, faces_list])

            # Get the index of the match
            match_index = matches.index(True)
            # Get the name of the match
            registered_faces = list(registered_students.keys())[match_index]

            current_time = datetime.now().strftime(" %Y-%m-%d %H:%M:%S ")
            match_info.append([registered_faces, path_cuadro, current_time])

# Show image
plt.imshow(frame)
plt.show()
print("Reporte de asistencia con reconocimiento facial: ")
print(registered_faces)

i += 1


In [ ]:
def check_if_face_registered(face_encoding):
  for _, registered_face in registered_faces.items():
    if True in face_recognition.compare_faces([face_encoding], registered_face):
      return True
    return False

###Consultas


In [ ]:
# Función para mostrar la información de profesores
def mostrar_info_profesores():
    for profesor in registered_professors:
        print(profesor)

# Función para mostrar la información de estudiantes
def mostrar_info_estudiantes():
    for estudiante in registered_students:
        print(estudiante)

# Función para mostrar la información de misceláneo
def mostrar_info_miscelaneo():
    for misc in registered_miscellaneous:
        print(misc)
print(registered_miscellaneous)

# Función para mostrar la información de aulas
def mostrar_info_aulas():
    for aula in aula_informacion:
        print(aula)

# Función para mostrar la información de cursos
def mostrar_info_cursos():
    for curso in informacion_curso:
        print(curso)
        # Aquí puedes buscar la información de profesores y estudiantes según sea necesario
        # y mostrarla junto con la información del curso
def asitencia_curso():
  from datetime import datetime

# Datos de la asistencia
#fecha_hora_reporte = datetime.now(_)
personas_presentes = ["Estudiante1", "Profesor1", "Misceláneo1"]

# Recorre las personas presentes en la asistencia
for persona in personas_presentes:
    # Obtén la información de la persona
    if persona in registered_students:
        informacion_persona = registered_students[persona]
    elif persona in registered_professors:
        informacion_persona = registered_professors[persona]
    elif persona in registered_miscellaneous:
        informacion_persona = registered_miscellaneous[persona]
    else:
        # La persona no se encontró en ningún diccionario.
        informacion_persona = {"Nombre": "Desconocido", "OtraInformacion": "N/A"}

    # Combina la información con la fecha y hora del reporte de asistencia
   # informacion_persona["FechaHoraReporte"] = fecha_hora_reporte

    # Muestra la imagen con un cuadro y el nombre de la persona
    print(f"Persona: {informacion_persona['Nombre']}")
    #print(f"Fecha y Hora de Reporte: {informacion_persona['FechaHoraReporte']}")

def ordenar_estudiantes_por_edad(students):
    n = len(registered_students)
    for i in range(n - 1):
        for j in range(0, n - i - 1):
            if students[j]['edad'] > students[j + 1]['edad']:
                # Intercambiar estudiantes[j] y estudiantes[j+1]
                students[j], students[j + 1] = students[j + 1], students[j]

# Llama a la función para ordenar a los estudiantes por edad
ordenar_estudiantes_por_edad(registered_students)
#print(registered_student)
# Imprimir la lista de estudiantes ordenada por edad
for estudiante in registered_students:
    print(f"Nombre: {estudiante['nombre']}, Edad: {estudiante['edad']}")

def ordenar_estudiantes_por_edad_descendente(students):
    # Obtén una lista de tuplas donde cada tupla contiene el nombre del estudiante y su edad
    student_list = [(name, student['informacion_adicional']['edad']) for name, student in students.items()]

    # Ordena la lista de tuplas por la edad de forma descendente
    for i in range(1, len(student_list)):
        current_student = student_list[i]
        j = i - 1
        while j >= 0 and current_student[1] > student_list[j][1]:
            student_list[j + 1] = student_list[j]
            j -= 1
        student_list[j + 1] = current_student

    # Crea un nuevo diccionario ordenado
    sorted_students = {name: students[name] for name, _ in student_list}

    return sorted_students

# Llama a la función para ordenar a los estudiantes por edad de forma descendente
students_ordenados_descendente = ordenar_estudiantes_por_edad_descendente(registered_students)

# Imprime el resultado
for name, student in students_ordenados_descendente.items():
    print("Nombre:", name)
    print("Edad:", student['informacion_adicional']['edad'])
    print("Carrera:", student['informacion_adicional']['carrera'])
    print()

# Menú principal
while True:
    print("Menú Principal:")
    print("1. Mostrar información de profesores")
    print("2. Mostrar información de estudiantes")
    print("3. Mostrar información de misceláneo")
    print("4. Mostrar información de aulas")
    print("5. Mostrar información de cursos")
    print("6. Consulta de asistencia por curso")
    print("7. Consulta de estudiantes por edad ascendente")
    print("8. Consulta de estudiantes por edad descendente")
    print("9. Consulta de estudiante que asistió más veces")
    print("10. Consulta de estudiante que asistió menos veces")
    print("11. Consulta de momento con más asistencias en un curso")
    print("12. Salir")

    opcion = input("Selecciona una opción: ")

    if opcion == "1":
        mostrar_info_profesores()
    elif opcion == "2":
        mostrar_info_estudiantes()
    elif opcion == "3":
        mostrar_info_miscelaneo()
    elif opcion == "4":
        mostrar_info_aulas()
    elif opcion == "5":
        mostrar_info_cursos()
    elif opcion == "6":

        pass
    elif opcion == "7":
        # Implementa la consulta de estudiantes por edad ascendente
        pass
    elif opcion == "8":
        # Implementa la consulta de estudiantes por edad descendente
        pass
    elif opcion == "9":
        # Implementa la consulta de estudiante que asistió más veces
        pass
    elif opcion == "10":
        # Implementa la consulta de estudiante que asistió menos veces
        pass
    elif opcion == "11":
        # Implementa la consulta de momento con más asistencias en un curso
        pass
    elif opcion == "12":
        break
    else:
        print("Opción no válida. Por favor, selecciona una opción válida.")
